# CNN (Convolutional Neural Network)
이번 실습에서는 텐서플로우를 사용하여 CNN을 구현해보도록 하겠습니다.  
실습 데이터로는 MNIST 데이터를 사용합니다.

In [1]:
from IPython.display import Image

## CNN이 이미지 처리에 자주 사용되는 이유
CNN의 장점은 가장 기본적인 딥러닝 모델인 다층퍼셉트론과 비교하면 쉽게 이해 됩니다.  
아래 다층퍼셉트론으로 MNIST 데이터를 입력 받아 숫자를 예측하는 모델을 보도록 하겠습니다.

In [2]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/mlp_overview.png", width=500, height=250)

다층퍼셉트론의 첫번째 단점은 고유 이미지의 생김새 정보를 사용할 수 없는 것입니다. 사람이 숫자의 생김새를 봤을 때, 직선이 위에서 아래로 하나 있으면 숫자 1, 동그라미 한개가 있으면 숫자 0이라는 정보를 쉽게 알 수 있습니다. 다층퍼셉트론의 경우, 데이터를 입력하기 위해서, 2차원 평면에 있는 숫자를 1차원 배열로 형변환시켜야합니다. 1차원 배열로 형변환된 데이터는 위 그림에서 보실 수 있듯이, 언뜻 사람이 이 형변환된 배열을 봐도, 원래 고유의 숫자 이미지를 떠올리기 쉽지 않습니다. 이는 컴퓨터도 마찬가지입니다. 형변환된 데이터를 입력받는 다층퍼셉트론의 경우, 고유 데이터가 2차원 평면에서 가지고 있던 지역적인 정보를 잃은 채로 학습을 시작합니다.  

In [3]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/mlp_overview2.png", width=500, height=250)

두번째 단점은 다층퍼셉트론은 픽셀 하나하나의 변화에 상당히 민감하다는 것입니다. 이미지의 생김새로 이미지를 분류를 할 경우, 픽셀 한두개가 달라도, 작대기 하나만 있는 숫자는 1, 동그라미 1개만 있는 숫자는 0이라고 쉽게 분류할 것입니다. 하지만 이미지의 생김새 정보를 사용할 수 없는 다층퍼셉트론은, 가지고 있는 정보가 픽셀들밖에 없기 때문에, 픽셀 한두개의 차이가 모델 예측에 큰 영향을 끼치게 됩니다.  

셋째 단점은, 픽셀 한두개의 정보에도 민감하기에, 다층퍼셉트론은 상당히 많은 변수를 내포하고 있습니다. 이는 모델의 사이즈를 크게 만들고, 학습 시간이 오래 걸리며, 자칫 잘못하면 과대적합된 모델이 되기 쉽습니다.

## 생김새 정보 획득하기
다층퍼셉트론의 단점은 최초부터 생김새 정보를 잃고 시작하는 데서 비롯됩니다. 그렇다면 어떻게 하면 모델이 생김새 정보를 잃지 않고 활용할 수 있을까요? 우리의 뇌는 어떻게 생김새 정보를 활용하여 사물을 구별할까요? 아래 그림을 보시면 두개의 비슷한 생김새의 숫자 2를 보실 수 있습니다. 사람의 뇌는 숫자 2는 머리 부분과 꼬리 부분이 있고, 머리와 꼬리를 연결시키는 대각선의 선이 있다는 것을 어렸을 때부터 학습받았기 때문에, 머리, 꼬리, 대각선 등의 생김새를 보면, 단번에 숫자 2임을 판별해냅니다.

In [4]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/cnn_filter.png", width=500, height=250)

## 생김새 정보를 학습하는 CNN
CNN은 바로 우리의 뇌가 사물을 구별하듯, 생김새 정보로 사물을 학습하고 구별해냅니다. 생김새는 각 사물의 특징(feature)라고 할 수 있고, CNN은 학습 과정을 통해 어떠한 특징이 사물을 구별하는 주요 특징인지를 알아냅니다.

## CNN은 어떻게 특징을 찾아내는가?
다층퍼셉트론과 다르게, CNN은 고유 이미지의 형변환 없이 그대로 입력 데이터를 처리합니다. 즉 2차원 데이터인 MNIST 데이터 그대로 처리하기 때문에, 생김새 정보를 잃지 않습니다.

아래 그림은 CNN이 어떻게 특징을 찾아내는 가를 잘 설명해줍니다.

In [5]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/stride.png", width=500, height=250)

### stride, filter, receptive field
위 그림에서 필터 또는 커널이라고 불리우는 반투명한 네모 상자가 이미지의 왼쪽 최상단에서 조금씩 이동하여, 최종적으로 오른쪽 최하단까지 이동하는 것을 볼 수 있습니다. 이렇게 필터를 조금씩 이동하는 기법을 stride라고 합니다.

필터는 특징을 추출하기 위한 네모 상자입니다. 그리고 이 필터와 겹치는 이미지의 부분을 receiptive field라고 부릅니다. 위 그림의 예제에서는 대각선 생김새를 찾기 위한 필터가 stride하는 모습을 보여줍니다.

아래 그림에서, 대각선 필터는 숫자 2로부터 두 곳의 대각선 특징을 감지해냅니다.
반면에 숫자 1에서는 한 곳의 대각선 특징도 발견하지 못합니다.

In [6]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/filter_diff.png", width=500, height=250)

모든 딥러닝 모델이 그렇듯, CNN 모델 역시 수학적인 모델입니다. 이제 필터가 어떻게 특징을 추출해내는 지, 아주 간단한 산수로 알아보도록 하겠습니다. MNIST 숫자 데이터는 흑백 이미지로 각각의 픽셀은 숫자 0부터 255까지 중 하나의 값을 가지고 있습니다. 픽셀 안의 숫자 0은 흰색을 의미하며, 255는 검은색, 그리고 그 안의 숫자는 흰색과 검은색 사이의 어떤 색을 의미합니다. 만약 이미지 안에 최고로 흰색과 최고로 검은색만 존재한다면, 아래 그림과 같이 0과 255로만 구성되어 있을 것입니다.

In [7]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/elem_mul.png", width=500, height=250)

위 그림에서 알 수 있듯, 필터 안에는 특정한 숫자가 들어있고, 필터와 이미지 영역의 겹치는 부분과 맞닿는 부분마다 곱셈이 이뤄집니다. 필터에 총 4개의 픽셀이 있으면 4개의 영역이 곱해지고, 곱해진 값은 최종적으로 더해집니다. 최종적으로 더해진 값이 크다라는 의미는, 필터와 겹쳐진 부분이 많았다라는 의미이고, 반대로 수치가 작을 경우, 필터와 겹치는 부분이 적었다라는 의미입니다.

숫자를 구분하기 위해서, CNN 모델 안에는 여러개의 필터가 존재해야합니다. 마치 우리 뇌가 숫자를 인식하기 위해서 직선, 곡선, 동그라미 등을 구별해야 하듯, CNN 모델 안에는 각각의 특성의 갯수만큼의 필터가 필요합니다. 
보통 앞쪽에 있는 레이어에서 직선, 곡선 같은 기초적인 특징을 구별하는 필터들이 존재하고, 뒤쪽에 있는 레이어에서는 동그라미, 세모 같은 조금 더 고차원의 특징을 구별하는 필터가 존재하게 됩니다.
필터를 사용하여 모든 특징을 골라낸 후, 이 특징들은 모든 노드들이 연결된 레이어(fully connected layer, FC)들의 입력값으로 들어가서 필터로부터 구별된 특징을 기반으로 한 숫자에 대한 예측을 진행합니다.  

Fully Connected Layer는 사실 다층퍼셉트론과 구조적으로는 다를 게 없습니다. 다만, 다층퍼셉트론은 픽셀의 값을 특징으로 사용한 반면, CNN의 Fully Connected Layer는 생김새(필터가 구별한 특징)을 특징으로 사용한다는 차이만이 있을 뿐입니다.

아래 CNN의 구조 및 필터의 역할을 잘 설명하는 그림을 참조하겠습니다.

In [8]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/cnn_architecture.png", width=500, height=250)

citation:
source from http://cs231n.github.io/convolutional-networks/

위 그림에서 CONV는 지금까지 제가 설명드렸던 필터가 존재하는 convolutional layer를 뜻합니다. 또한 가장 마지막에 위치한 FC는 Fully Connected Layer를 의미합니다. 보통 CNN에서는 그림과 같이 Relu 활성화함수를 많이 사용합니다. 이번장의 실습에서도 마찬가지로 Relu를 CNN의 활성화함수로 사용하도록 하겠습니다.

위 그림에서 POOL은 풀링레이어(pulling layer)이며, 풀링 레이어의 주된 목적은 모델 파라미터를 줄임으로써, 모델의 계산량을 줄이는 데 있습니다. 파라미터가 줄어드니, 오버피팅을 조절하는 효과도 기대할 수 있습니다

위 그림에서 다뤘던 숫자 2의 예제를 다시 사용해보도록 하겠습니다. 스트라이드를 통해 아래의 그림과 같이 4 * 4의 행렬을 얻게됩니다. 스트라이드를 통해 얻어진 행렬을 피쳐맵(feature map)이라고 부릅니다. 참고로 피쳐맵을 활성화함수에 넣어 얻어진 행력을 액티베이션맵(activation map)이라고도 부릅니다. 아래 그림은 스트라이드를 통해 피쳐맵을 계산하는 과정을 보여줍니다.

In [9]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/stride_result.png", width=500, height=250)

피쳐맵에 직접 풀링레이어를 적용하는 것을 가정해보겠습니다. 풀링은 주로 맥스 풀링 또는 평균 풀링을 사용합니다. 이번 예제에서는 맥스 풀링을 보여주고 있습니다. 맥스 풀링은 지정된 영역에서 가장 큰 수치를 선택하고 나머지 값은 버립니다. 아래 그림에서 2 * 2 필터에 스트라이드 2를 적용한 맥스풀링의 예를 보여주고 있습니다.

In [10]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/max_pool.png", width=500, height=250)

위 그림을 통해, 결과적으로 4 * 4 행렬이 맥스 풀링의 결과 2 * 2 행렬로 크기가 줄어든 것을 확인하실 수 있습니다. 피쳐맵의 크기가 줄어듬으로 인해 얻는 장점으로 첫째, 계산에 사용될 파라미터의 갯수가 줄어들어 계산 속도가 빨라지고, 둘째, 파라미터를 줄임으로써 모델의 분산을 줄이고, 그에 따라 과대적합 가능성을 줄여줍니다.

### 제로 패딩
마지막으로 제로 패딩(zero padding)이라는 개념을 소개해드리겠습니다. 제로 패딩이란 0으로 입력 행렬의 테두리를 감싸는 기술입니다. 제로 패딩은 최근 CNN 모델에 상당히 많이 사용되고 있으며, 사용 목적은 크게 두가지입니다.  

첫째, 제로 패딩은 convolutional layer에 의한 정보 손실을 방지합니다. convolutional layer에서 보통 스트라이딩을 통해 보통 입력된 행렬보다 작아진 행렬이 출력됩니다. 행렬이 작아지면 당연히 작아진만큼의 정보손실이 있습니다. 제로 패딩을 통해 입력 행렬의 사이즈를 크게함으로써, 스트라이딩 이후에도 그 사이즈를 동일하게 유지할 수 있습니다.  

둘째, 0으로 감싸진 부분이 테두리라는 정보 역시 CNN이 활용할 수 있는 정보입니다. 제로 패딩을 사용함으로써, CNN이 테두리 정보를 활용할 수 있게됩니다.

아래 그림의 예제를 보면서 이해해보도록 하겠습니다. 최초 5 X 5 행렬이 스트라이딩을 통해 4 X 4 피쳐맵으로 출력됩니다. 행렬의 크기가 줄었으니, 기존의 정보가 함축되었다 또는 줄어들었다고 볼 수 있습니다.

In [11]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/zeropadding1.png", width=500, height=250)

이번에는 제로 패딩이 적용된 예제를 보도록 하겠습니다. 제로패딩으로 인해 스트라이딩할 공간이 더 많아졌습니다. 만약 필터의 사이즈가 3 X 3이고 스트라이딩을 1 픽셀씩할 경우, 피쳐맵은 5 X 5로 제로패딩을 하기 전 입력 행렬의 사이즈와 동일한 사이즈로 출력될 것입니다.

In [12]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/zeropadding.png", width=500, height=250)

## CNN으로 컬러 이미지 분류하기

MNIST 손글씨 데이터는 흑백 이미지입니다. 만약 컬러 이미지를 데이터로 사용할 경우, 한가지만 더 신경써주시면 됩니다. 컬러 이미지는 단순히 RGB 즉, 빨강, 녹색, 파란색 레이어가 겹쳐진 색상입니다. 각각의 색상은 흑백과 마찬가지로 0부터 255까지의 값을 지니고 있습니다. 예를 들어 흑백 이미지와 컬러 이미지의 텐서 형태를 비교해보면, 가로 28 픽셀, 세로 28 픽셀의 흑백 이미지 10장을 처리할 경우, 텐서는 (10,28,28,1)의 형태를 갖고 동일한 크기의 컬러 이미지는 (10,28,28,3)의 텐서 형태를 갖습니다.

In [13]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/rgb.png", width=500, height=250)

위 그림에서 보시듯이, 컬러 이미지는 0부터255까지의 값을 지닌 빨강, 녹색, 녹색 레이어가 겹쳐서 보이는 색상에 불과합니다. CNN에서 이 겹쳐진 레이어의 갯수를 깊이(depth)라고 부릅니다. 아래 그림을 통해 하나의 피러가 어떻게 세 색상 레이어에 적용되는 지 보도록 하겠습니다. 하나의 필터에는, depth만큼의 서브 필터가 존재하고 서브필터는 각각 자기만의 파라미터를 가지고 있습니다. 아래 예제에서 대각선 필터의 예를 보도록 하겠습니다. 빨강 레이어에는 빨강 필터, 녹색 레이어에는 녹색 필터, 파랑 레이어에는 파랑 필터가 스트라이딩을 하며 각각의 피쳐맵을 만들고, 세개의 피쳐맵이 더해진 후 편향값까지 더해저셔 하나의 피쳐맵이 완성됩니다.

In [14]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/rgb1.png", width=500, height=250)

In [15]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/rgb2.png", width=500, height=250)

위 그림에서 알 수 있듯, 또다른 필터 역시 depth와 동일한 갯수의 서브필터를 지니고, 각각의 서브필터는 자신만의 파라미터를 갖습니다. 한개의 필터는 결과적으로 하나의 피쳐맵을 출력하므로, 만약 10개의 필터가 한 컨볼루셔널 레이어에 있다면, 다음 레이어에 입력되는 depth는 10이 됩니다.

## CNN 모델 학습 이해하기
아래 그림을 통해, 컬러 이미지로 된 손글씨 예측 CNN 모델을 알아보겠습니다.

In [16]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/cnn_train.png", width=500, height=250)

위 그림처럼, 만약에 입력 데이터가 컬러이미지일 경우, depth는 3이 되며, 그림에서와 같이 빨강 레이어, 녹색 레이어, 파랑 레이어 세개가 입력데이터로 들어가게 됩니다. 첫번째 CONV(convolutional layer)에 4개의 필터가 존재하며, 그림에서 CONV1의 출력값으로 4개의 피쳐맵이 출력되는 것을 볼 수 있습니다. 두번째 CONV에는 총 3개의 필터가 존재하고 세개의 피쳐맵이 출력됩니다. 그 뒤에 있는 풀링 레이어에서 피쳐맵의 크기가 절반으로 줄어들어 2 X 2의 형태의 피쳐맵이 3개 존재하는 것을 확인할 수 있습니다. 풀링레이어 이후의 모든 값을 일렬로 졍렬시켜서 한개의 배열에 총 2 X 2 X 3개의 값이 있는 것을 확인 할 수 있습니다. 이 12개의 값은 CONV로 획득된 특징(feature)들이며, 특징들을 FC(fully connected layer)에 연결시켜 숫자 0부터 9까지의 값을 상징하는 10개의 노드로 값을 전달한 후, 소프트맥스를 통해 각 숫자에 대한 확률을 출력합니다. 옵티마이저를 사용하여 역전파를 통해 각 레이어에 존재하는 파라미터를 최적화합니다.

### 파라미터 최적화
학습의 목적은 각 레이어에 존재하는 파라미터를 최적화하는 것입니다. CNN은 CONV레이어와 FC레이어에 파라미터가 존재합니다. CONV레이어는 특징을 추출하는 레이어이고, FC레이어는 추출된 특징을 가지고 분류를 수행하는 레이어입니다. 학습 시작 시, 랜덤하게 파라미터의 값을 부여하지만, 학습 과정에서 진행되는 역전파를 통해 각각의 파라미터가 조금씩 변경되며 알맞은 값으로 최적화됩니다.  

CNN의 필터들에게 역할을 부여하는 것은 누구일까요? 실습 과정에서 보시겠지만, 설계자는 필터의 갯수와 형태만 정할 뿐, A필터는 대각선 필터, B필터는 동그라미 필터라는 식으로 역할을 정해주지 않습니다. 필터의 역할은 최적화 과정을 통해 비지도 학습의 방법으로 모델이 스스로 찾아 부여하게 됩니다.  

CNN 모델의 예측값과 데이터의 실제값의 차이를 줄여나가는 방향으로 모델을 최적화함으로써, 보다 의미있는 필터(CONV)가 만들어지고, 보다 의미 있는 분류기(FC)가 완성이 됩니다.  

보통 CNN은 경사하강법(gradient descent) 방식으로 모델을 최적화합니다. 아래 그림에서 Loss는 예측값과 실제값의 차이이고, W는 모델의 파라미터입니다. Loss가 가장 낮은 부분은 아래 그래프에서 미분값이 0인 부분이므로, 현재 W인 지점에서의 미분값을 구해서 그 미분값이 양수면 왼쪽 방향으로, 음수면 오른쪽 방향으로 W 값을 조금씩 이동하여, 미분값이 최저가 되는 곳으로 W를 조정해줍니다. 

In [17]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/sgd.png", width=300, height=150)

## 텐서플로우로 CNN 실습하기
텐서플로우로 아래 그림과 동일한 CNN을 직접 구현하고, MNIST 손글씨로 학습 및 테스트를 진행보도록 하겠습니다.

In [18]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/practice_cnn.png", width=800, height=200)

In [19]:
import tensorflow as tf

### 데이터 획득

In [20]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

학습 데이터는 총 **60000** 개의 샘플이 있습니다.  
테스트 데이터는 총 **10000** 개의 샘플이 있습니다.     
모든 데이터는 **28 * 28** 의 픽셀을 가지고 있습니다.  

MNIST 데이터는 아래의 그림처럼, 28*28의 픽셀 데이터입니다.  
각각의 픽셀은 흑백 사진과 같이 0부터 255까지의 그레이 스케일을 가지고 있습니다.

![0 from MNIST](https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/mnist_sample.png)

### 학습데이터에서 검증 데이터 분리하기
학습 중간마다 검증 데이터로 모델의 성능을 측정하면 다음과 같은 장점이 있습니다.  
1) 모델 학습이 제대로 진행되는 지 검증 정확도로 확인할 수 있습니다.  
2) 학습 정확도는 올라가는 데 검증 정확도가 안올라가거나 떨어질 시, 조기 종료를 구현할 수 있습니다.

In [21]:
x_val  = x_train[50000:60000]
x_train = x_train[0:50000]
y_val  = y_train[50000:60000]
y_train = y_train[0:50000]

In [22]:
print("train data has " + str(x_train.shape[0]) + " samples")
print("every train data is " + str(x_train.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

train data has 50000 samples
every train data is 28 * 28 image


In [23]:
print("validation data has " + str(x_val.shape[0]) + " samples")
print("every train data is " + str(x_val.shape[1]) 
      + " * " + str(x_train.shape[2]) + " image")

validation data has 10000 samples
every train data is 28 * 28 image


**0** 부터 **255** 까지의 그레이 스케일을 확인할 수 있습니다.

In [24]:
# sample to show gray scale values
print(x_train[0][8])

[  0   0   0   0   0   0   0  18 219 253 253 253 253 253 198 182 247 241
   0   0   0   0   0   0   0   0   0   0]


**0** 부터 **9**까지의 이미지에 해당하는 숫자를 확인할 수 있습니다.

In [25]:
# sample to show labels for first train data to 10th train data
print(y_train[0:9])

[5 0 4 1 9 2 1 3 1]


테스트 데이터는 **10000** 개의 샘플을 가지고 있습니다.  
모든 테스트 데이터는 **28 * 28** 의 이미지입니다.

In [26]:
print("test data has " + str(x_test.shape[0]) + " samples")
print("every test data is " + str(x_test.shape[1]) 
      + " * " + str(x_test.shape[2]) + " image")

test data has 10000 samples
every test data is 28 * 28 image


### 데이터 구조 변경하기
다층퍼셉트론의 입력 레이어에 데이터를 넣기 위해서 데이터의 구조를 변경해줍니다.

In [27]:
import numpy as np
x_train = np.reshape(x_train, (50000,28,28,1))
x_val = np.reshape(x_val, (10000,28,28,1))
x_test = np.reshape(x_test, (10000,28,28,1))

print(x_train.shape)
print(x_test.shape)

(50000, 28, 28, 1)
(10000, 28, 28, 1)


### 데이터 정규화
데이터 정규화는 보통 학습 시간을 단축하고, 더 나은 성능을 구하도록 도와줍니다.  
MNIST 데이터의 모든 값은 0부터 255의 범위 안에 있으므로, 255로 값을 나눠줌으로써, 모든 값을 0부터 1 사이의 값으로 정규화합니다.  

In [28]:
x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

gray_scale = 255
x_train /= gray_scale
x_val /= gray_scale
x_test /= gray_scale

### 실제값을 one hot encoding으로 변경하기
손실 함수에서 크로스 엔트로피를 계산하기 위해, 실제값은 one hot encoding 값으로 변경합니다.

In [29]:
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

### CNN 텐서플로우로 구현하기

In [30]:
Image(url= "https://raw.githubusercontent.com/minsuk-heo/deeplearning/master/img/practice_cnn.png", width=800, height=200)

MNIST 데이터 형태 그대로 28 X 28의 포맷을 그대로 입력 데이터로 사용합니다.  
타겟은 0부터 9까지의 숫자입니다.

In [31]:
x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

다음으로 파라미터(weight, bias) 초기값을 설정해줍니다. 아래 코드에서 tf.truncated_normal은 0에서부터 거리가 아주 먼 값이 설정되지 않도록 도와줍니다. 0에서부터 거리가 먼 값을 제외하는 이유는, 소프트맥스 안에 있는 시그모이드 함수의 특성 때문입니다. 시그모이드 같은 경우, 입력값이 매우 크거나 작으면 그 미분값이 0과 가까워져서 경사하강법으로 파라미터를 변경시키기 어려워집니다.

In [32]:
def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

padding='SAME'은 레이어에 입력된 피쳐맵의 사이즈와 동일하게 피쳐맵을 출력하도록 설정해줍니다.
예를 들어, CONV1에 28 X 28의 이미지 데이터가 입력되면, 동일한 크기인 28 X 28 형태의 피쳐맵이 출력되게 됩니다.

In [33]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

첫번째 CONV 레이어는 총 16개의 필터를 가지고 있고 필터의 사이즈는 5 X 5입니다.
편향값(bias)는 필터의 갯수만큼 만들어줍니다.

In [34]:
W_conv1 = weight_variable([5, 5, 1, 16])
b_conv1 = bias_variable([16])

활성화함수로 relu를 사용합니다.

In [35]:
h_conv1 = tf.nn.relu(conv2d(x, W_conv1) + b_conv1)

CONV 레이어 다음으로 풀링 레이어를 적용하여 activation map의 크기를 줄여줍니다. activation map의 크기를 줄여줌으로써, 파라미터가 줄어들어 모델 크기가 작아지고, 과대적합의 위험도 감소시켜줍니다.

In [36]:
h_pool1 = max_pool_2x2(h_conv1)

풀링레이어의 영향으로, activation map의 사이즈는 14 X 14가 되었습니다. 이 값은 다음에 이어지는 CONV2의 입력으로 들어가게 됩니다. CONV2는 총 32개의 필터를 가지고 있습니다.

In [37]:
W_conv2 = weight_variable([5, 5, 16, 32])
b_conv2 = bias_variable([32])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

풀링레이어 이후의 activation map의 사이즈는 7 X 7이 됩니다.  

#### FC (Fully Connected Layer)
FC는 CONV를 통해 추출된 이미지의 특징들을 입력으로 받아 0부터 9까지의 숫자 중 하나로 이미지를 분류합니다.
아래 코드에서 tf.reshape를 사용하여 모든 특징들을 하나의 배열로 형변환합니다. 형변환된 특징들은 FC의 입력값이 됩니다.

In [38]:
W_fc1 = weight_variable([7 * 7 * 32, 128])
b_fc1 = bias_variable([128])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

FC 영역에는 FC1, FC2 총 2개의 히든 레이어가 존재합니다. FC1은 128개의 노드, FC2에는 10개의 노드가 존재합니다. FC2에 10개의 노드가 존재하는 이유는 FC2의 10개 노드의 값들을 소프트맥스에 입력시켜서 각 노드별 확률을 구하기 위해서입니다. 각 노드는 숫자 0부터 9를 의미하며, 이 예측값은 크로스 엔트로피를 통해 실제값과의 차이를 계산하는 데 사용됩니다.

In [39]:
W_fc2 = weight_variable([128, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1, W_fc2) + b_fc2

아래와 같이 크로스엔트로피를 설정해줍니다.

In [40]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y_conv))

아담 옵티마이저를 사용해서 모델을 최적화합니다. 학습률은 0.001로 설정하였습니다.

In [41]:
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

아래 코드는 정확도를 구하기 위해서 사용합니다.

In [42]:
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

### 학습 및 테스트
이제 CNN 모델을 MNIST 데이터로 학습해보도록 하겠습니다. 단 3번의 주기만으로도 놀라온 정확도가 확인됩니다. 이번 실습에서는 미니배치를 사용하여 모델을 최적화합니다. 미니배치는 총 500개의 학습데이터가 들어있고, 미니배치마다 파라미터를 조정해줍니다.

In [43]:
# initialize
init = tf.global_variables_initializer()

# train hyperparameters
epoch_cnt = 3
batch_size = 500
iteration = len(x_train) // batch_size

# Start training
with tf.Session() as sess:
    tf.set_random_seed(777)
    # Run the initializer
    sess.run(init)
    for epoch in range(epoch_cnt):
        avg_loss = 0.
        start = 0; end = batch_size
        
        for i in range(iteration):
            if i%10 == 0:
                train_accuracy = accuracy.eval(feed_dict={x:x_train[start: end], y_: y_train[start: end]})
                print("step "+ str(i) + ": training accuracy: "+str(train_accuracy))
            train_step.run(feed_dict={x:x_train[start: end], y_: y_train[start: end]})
            start += batch_size; end += batch_size    
        
        # Validate model
        val_accuracy = accuracy.eval(feed_dict={x:x_val, y_: y_val})
        print("validation accuracy: "+str(val_accuracy))
        
    test_accuracy = accuracy.eval(feed_dict={x:x_test, y_: y_test}) 
    print("test accuracy: "+str(test_accuracy))

step 0: training accuracy: 0.11
step 10: training accuracy: 0.628
step 20: training accuracy: 0.776
step 30: training accuracy: 0.854
step 40: training accuracy: 0.896
step 50: training accuracy: 0.91
step 60: training accuracy: 0.906
step 70: training accuracy: 0.92
step 80: training accuracy: 0.934
step 90: training accuracy: 0.936
validation accuracy: 0.9475
step 0: training accuracy: 0.946
step 10: training accuracy: 0.948
step 20: training accuracy: 0.94
step 30: training accuracy: 0.95
step 40: training accuracy: 0.952
step 50: training accuracy: 0.958
step 60: training accuracy: 0.964
step 70: training accuracy: 0.96
step 80: training accuracy: 0.97
step 90: training accuracy: 0.966
validation accuracy: 0.9712
step 0: training accuracy: 0.966
step 10: training accuracy: 0.972
step 20: training accuracy: 0.96
step 30: training accuracy: 0.972
step 40: training accuracy: 0.976
step 50: training accuracy: 0.978
step 60: training accuracy: 0.98
step 70: training accuracy: 0.972
step

단 **3** 번의 주기만으로도, **97.8%** 의 정확도를 확인할 수 있습니다.